# AL Documentation

## Introduction

AL simplifies the complexity of large language models into a simple, open-source solution. One intuitive interface. Multiple AI powerhouses. Ollama, Anthropic, OpenAI - seamlessly integrated into all python environments, you can use it even on 1970 terminals if youd like that sort of a thing.
AL doesn't just fetch answers; it preserves your learning journey. Every interaction, every spark of inspiration, is captured in Jupyter notebooks. Share, version, and automate your AI workflows with ease.

But here's the game-changer: these saved sessions aren't mere logs. They're goldmines for AI advancement. Use them to fine-tune models, iterate rapidly, and fuel your learning. Reuse, edit, and evolve your AI interactions effortlessly.
* simple ASCII/md interface
* model selection
* save/load history as .ipynb file

## Why Use AL?

AL offers several advantages over other similar tools:

1. **Unified Interface**: AL provides a consistent interface for interacting with different AI models, simplifying the process of switching between or comparing various AI services.

2. **Extensibility**: The base class is designed to be easily extended, allowing developers to add support for new AI services by creating subclasses.

3. **History Management**: AL includes built-in conversation history management, making it easy to maintain context in multi-turn conversations.

4. **Code Handling**: The class includes methods for managing and setting code snippets to the clipboard, which is particularly useful for programming-related tasks.

5. **Jupyter Notebook Integration**: AL supports saving and loading conversation history in Jupyter Notebook format, facilitating easy documentation and sharing of AI interactions.

## AL Class Overview

1. The `AL` class serves as the foundation for AI model interactions. Here's an overview of its main components:

```python
class AL:
    def __init__(self, interface: Literal["ollama", "claude", "chatgpt"] = "ollama", model: str = "llama3.1", api_key: Optional[str] = None):
    # implementation

    def ask(self, question):
    # implementation
```

2. To acess the code snipets from the answer:
```python
    def copy_code(def copy_code(self, index: Optional[int] = None):
        # If no index then last code snippet is copied, otherwise the index runs from last to first
```

3. To store/edit as jupyter lab workbook
```python
        
    def save_history(self, filename):
        # Implementation for saving history to a Jupyter Notebook

    def load_history(self, filename):
        # Implementation for loading history from a Jupyter Notebook
```
## Conclusion

AL provides a powerful and flexible foundation for interacting with various AI language models. By using AL, developers can easily integrate different AI services into their projects, manage conversation history, and handle code snippets efficiently. The extensible nature of AL allows for easy addition of new AI services, making it a valuable tool for developers working with multiple AI platforms.

## Generic AL object definition

In [5]:
from typing import Literal, Optional, List, Dict
from IPython.display import Markdown
from datetime import datetime
import json
import nbformat as nbf
import pyperclip

class AL:
    def __init__(
        self, 
        interface: Literal["ollama", "claude", "chatgpt"] = "ollama",
        model: str = "llama3.1",
        api_key: Optional[str] = None,
        ollama_host: Optional[str] = None
    ):
        self.interface = interface
        self.model = model
        self.api_key = api_key
        self.ollama_host = ollama_host  # Store the Ollama host
        self.history = []
        self.last_code = []
        self.active_code = 0
        
        # Initialize appropriate client
        self._init_client()

    def _init_client(self):
        if self.interface == "ollama":
            import ollama
            if self.ollama_host:
                ollama.set_host(self.ollama_host)  # Set custom Ollama host if provided
            self.client = ollama
        elif self.interface == "claude":
            from anthropic import Anthropic
            self.client = Anthropic(api_key=self.api_key)
        else:  # chatgpt
            import openai
            self.client = openai.OpenAI(api_key=self.api_key)

    def split_response(self, response: str) -> List[Dict]:
        segments = []
        parts = response.split('```')
        
        for i, part in enumerate(parts):
            if i % 2 == 0:
                if part.strip():
                    segments.append({
                        "type": "markdown", 
                        "content": part.strip()
                    })
            else:
                lines = part.splitlines()
                if lines:
                    language = lines[0].strip()
                    code = '\n'.join(lines[1:])
                    self.last_code.insert(0, code.strip())
                    segments.append({
                        "type": "code", 
                        "language": language, 
                        "content": code.strip()
                    })
        
        return segments

    def ask(self, message: str):
        # Store user message
        self.history.append({
            "role": "human",
            "content": message,
            "timestamp": datetime.now().isoformat()
        })
        
        # Get response based on interface
        response = self._get_response(message)
        
        # Process and store response
        self.history.append({
            "role": "assistant",
            "content": response,
            "timestamp": datetime.now().isoformat()
        })
        
        # Display response
        segments = self.split_response(response)
        for segment in segments:
            if segment['type'] == 'markdown':
                display(Markdown(segment['content']))
            else:
                print(f"```{segment['language']}\n{segment['content']}\n```")
        
    # get last answer as string

    def _get_response(self, message: str) -> str:
        if self.interface == "ollama":
            response = self.client.chat(
                model=self.model, 
                messages=[{"role": "user", "content": message}]
            )
            return response['message']['content']
        elif self.interface == "claude":
            response = self.client.messages.create(
                model=self.model,
                messages=[{"role": "user", "content": message}]
            )
            return response.content[0].text
        else:  # chatgpt
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": message}]
            )
            return response.choices[0].message.content

    def copy_code(self, index: Optional[int] = None):
        """Copy specific code block to clipboard"""
        if index is None:
            index = self.active_code
        if 0 <= index < len(self.last_code):
            pyperclip.copy(self.last_code[index])
            print(f"Copied code block {index} to clipboard")
        else:
            print("Invalid code block index")

    def save_history(self, filename: str):
        """Save conversation to notebook"""
        if not filename.endswith('.ipynb'):
            filename += '.ipynb'
            
        nb = nbf.v4.new_notebook()
        cells = []

        for entry in self.history:
            # Add role cell
            cells.append(nbf.v4.new_markdown_cell(
                f"## {entry['role'].capitalize()}"
            ))

            # Split and add content cells
            segments = self.split_response(entry['content'])
            for segment in segments:
                if segment['type'] == 'markdown':
                    cells.append(nbf.v4.new_markdown_cell(
                        segment['content']
                    ))
                else:
                    cells.append(nbf.v4.new_code_cell(
                        segment['content']
                    ))

        nb['cells'] = cells
        
        with open(filename, 'w', encoding='utf-8') as f:
            nbf.write(nb, f)

    def load_history(self, filename: str):
        """Load conversation from notebook"""
        if not filename.endswith('.ipynb'):
            filename += '.ipynb'
            
        with open(filename, 'r', encoding='utf-8') as f:
            nb = nbf.read(f, as_version=4)
        
        self.history = []
        self.last_code = []
        
        current_entry = None
        
        for cell in nb.cells:
            source = cell['source']
            
            if cell['cell_type'] == 'markdown' and source.startswith('## '):
                # If we have a previous entry, add it
                if current_entry:
                    self.history.append(current_entry)
                
                # Start new entry
                role = source[3:].lower().strip()
                current_entry = {
                    'role': role,
                    'content': '',
                    'timestamp': datetime.now().isoformat()
                }
            elif current_entry:
                # Add content to current entry
                if cell['cell_type'] == 'code':
                    self.last_code.append(source)
                    current_entry['content'] += f'```python\n{source}\n```\n'
                else:
                    current_entry['content'] += f'{source}\n'
        
        # Add last entry if exists
        if current_entry:
            self.history.append(current_entry)

# Testing and ideas

In [6]:
chat = AL()

In [7]:
chat.ask("can you write simple random number generator in python")

Here is a simple implementation of a Random Number Generator (RNG) in Python:

```python
import secrets

class RNG:
    def __init__(self, seed=None):
        if seed is not None:
            self.seed = seed
        else:
            self.seed = secrets.randbelow(2**32)

    def random(self):
        return self.seed * 1103515245 + 12345 >> 16


# Example usage:

rng = RNG(seed=42)
print(rng.random())  # Output: a random number between 0 and 32767

for _ in range(10):
    print(rng.random())
```


However, the above implementation is not cryptographically secure. For security-critical applications (like password generation or cryptographic protocols), you should use the `secrets` module from Python's standard library:

```python
import secrets

def rng():
    return secrets.randbelow(2**32)


# Example usage:

print(rng())  # Output: a random number between 0 and 4294967295
```


The `secrets` module is designed to generate cryptographically strong random numbers suitable for managing data such as passwords, account authentication, security tokens, and related secrets.

In [21]:
chat.copy_code(1)

Copied code block 1 to clipboard


In [24]:
import random

def generate_random_number(min_val, max_val):
    """
    Generate a random integer between min_val and max_val.

    Args:
        min_val (int): The minimum value to generate.
        max_val (int): The maximum value to generate.

    Returns:
        int: A random integer between min_val and max_val.
    """
    return random.randint(min_val, max_val)

# Example usage
random_num = generate_random_number(1, 100)
print(f"Random number: {random_num}")

Random number: 92


In [17]:
chat.last_code[0]

'class SimpleRNG:\n    def __init__(self, seed):\n        self.seed = seed\n\n    def generate_random_number(self, min_val, max_val):\n        """\n        Generate a random integer between min_val and max_val.\n\n        Args:\n            min_val (int): The minimum value to generate.\n            max_val (int): The maximum value to generate.\n\n        Returns:\n            int: A random integer between min_val and max_val.\n        """\n        self.seed = (1103515245 * self.seed + 12345) % 2**31\n        return min_val + (max_val - min_val) * (self.seed // (2**31))\n\n# Example usage\nrng = SimpleRNG(42)\nrandom_num = rng.generate_random_number(1, 100)\nprint(f"Random number: {random_num}")'